In [ ]:
!pip install pyyaml==5.1
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import json
from pandas.io.json import json_normalize
import os
import torch
import copy

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T

from pycocotools.coco import COCO
import skimage.io as io
from detectron2.utils.visualizer import ColorMode

from PIL import Image
import glob
import pickle

### Data Path

In [ ]:
dataset_dir = '/content/drive/MyDrive/'
train_dir = "train/"
label_dir = "labels/"
labeled_dir = "labeled_images/"
test_dir = "test/images/"

In [ ]:
unlabel = glob.glob("/content/drive/MyDrive/train/unlabeled_images/"+"*.jpg")

In [ ]:
with open('/content/drive/MyDrive/dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)

with open('/content/drive/MyDrive/aug_data.pickle', 'rb') as f:
    aug_dataset = pickle.load(f) 

In [ ]:
classes = ['container_truck', 'forklift', 'reach_stacker', 'ship']

### Functions define

In [ ]:
import skimage.measure as measure

def close_contour(contour):
  if not np.array_equal(contour[0], contour[-1]):
    contour = np.vstack((contour, contour[0]))
  return contour

def binary_mask_to_polygon(binary_mask, tolerance=0):
  polygons = []
  # pad mask to close contours of shapes which start and end at an edge
  padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
  contours = measure.find_contours(padded_binary_mask, 0.5)
  contours = np.subtract(contours, 1)
  for contour in contours:
      contour = close_contour(contour)
      contour = measure.approximate_polygon(contour, tolerance)
      if len(contour) < 3: 
          continue
      contour = np.flip(contour, axis=1)
      segmentation = contour.ravel().tolist()
      # after padding and subtracting 1 we may get -0.5 points in our segmentation
      segmentation = [0 if i < 0 else i for i in segmentation]
      polygons.append(segmentation)

  return polygons


def mask_to_coordinates(mask):
    flatten_mask = mask.flatten()
    if flatten_mask.max() == 0:
        return f'0 {len(flatten_mask)}'
    idx = np.where(flatten_mask!=0)[0]
    steps = idx[1:]-idx[:-1]
    new_coord = []
    step_idx = np.where(np.array(steps)!=1)[0]
    start = np.append(idx[0], idx[step_idx+1])
    end = np.append(idx[step_idx], idx[-1])
    length = end - start + 1
    for i in range(len(start)):
        new_coord.append(start[i])
        new_coord.append(length[i])
    new_coord_str = ' '.join(map(str, new_coord))
    
    return new_coord_str


In [ ]:
obj = {}

def make_self_training_data(outputs,filename):
        record = {}
        objs = []

        img = Image.open(filename) #사이즈 알려고 이미지 불러옴
        
        record["file_name"] = filename
        record["height"] =  img.size[1] 
        record["width"] = img.size[0]

        
        mask = np.asarray(outputs['instances'].pred_masks[0].cpu()) # 마스크 데이터
        ins_coords = binary_mask_to_polygon(mask) # 폴리곤으로 만듦
        if len(ins_coords) != 0 :
          px = [a for a in ins_coords[0][0::2]]
          py = [a for a in ins_coords[0][1::2]]
          poly = [[x, y] for x, y in zip(px, py)]

          global obj

          obj = {
            "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
            "bbox_mode": BoxMode.XYXY_ABS,
            "segmentation": [poly],
            "category_id": classes.index(filename[59:-15]), 
            "iscrowd": 0
        }

        objs.append(obj)
        record["annotations"] = objs

        return record

### Dataset Setting

In [ ]:
with open('/content/drive/MyDrive/output/add_data.pickle', 'rb') as f:
    add_data = pickle.load(f) 

len(add_data) # output : 12
add_data[0].keys() # output : dict_keys(['file_name', 'height', 'width', 'annotations'])

In [ ]:
remove_name =[]

for item in add_data:
  remove_name.append(item['file_name'])

len(remove_name) # output : 12

In [ ]:
for remove_name1 in remove_name: # 중복 안 되게 unlabel에서 삭제
  if remove_name1 in unlabel:
    unlabel.remove(remove_name1)
  else:
    continue 

len(unlabel) # output : 1553

### Model Config Setting

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_3x.yaml"))
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = len(classes)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_3x.yaml")

In [ ]:
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 5000   # 학습 횟수
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.TEST.EVAL_PERIOD = 100
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
#model_dir = "output/KH_Aug_mask_rcnn_R_50_DC5_3x" #"mask_rcnn_R_50_DC5_3x_7000" # 초기 모델 폴더 이름
cfg.OUTPUT_DIR = "/content/drive/MyDrive/output/top12_mul2_step_self_0"

#dataset_dir+model_dir #dataset_dir+'output/' + model_dir
#os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") # predictor 생성
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
predictor = DefaultPredictor(cfg) # 초기모델 predictor 불러오기

## Self-Training

In [ ]:
flag = 1 #이미 1번 학습 된 모델로 하기 때문에 flag는 1부터 시작
self_model_dir = "top12_mul2_step_self_"
#cfg.OUTPUT_DIR = dataset_dir+'output/' + self_model_dir
#os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
print('*********************** start ***********************')

add_data_num = 24 # 이미 1번 학습 된 모델로 하기 때문에 이후 1 epoch 돌 때마다 데이터 추가
#add_data = [] 위에서 이미 불러왔음!

while True:

  objs = []
  scores = []
  

  for un in unlabel:
      record = {}
      im = cv2.imread(un)
      outputs = predictor(im)  
      if len(outputs['instances'].pred_boxes) == 0:
        continue
      else:
        dic = make_self_training_data(outputs,un)
        objs.append(dic)
        record['score'] = outputs['instances'].scores[0].tolist()
        record['class'] = outputs['instances'].pred_classes[0].tolist()
        scores.append(record)

  temp = pd.DataFrame(objs)
  temp2 = pd.DataFrame(scores)
  temp3 = pd.concat([temp,temp2],axis=1)
  df_sort_top = temp3.sort_values(by="score", ascending=False).head(add_data_num)
  #df_sort_group_top3 = temp3.sort_values(by="score", ascending=False).groupby("class").head(add_data_num)
  
  #idx_lst = df_sort_group_top3.index.tolist()
  idx_lst = df_sort_top.index.tolist()

  for idx in idx_lst:
    add_data.append(objs[idx])

  for remove_name in df_sort_top['file_name']: # 중복 안 되게 unlabel에서 삭제
    if remove_name in unlabel:
      unlabel.remove(remove_name)
    else:
      continue 

  add_data_num = 2 * add_data_num
  print("*********************** add_data 길이: ",len(add_data),"***********************")
  print("*********************** unlabel 길이: ",len(unlabel),"***********************")


  DatasetCatalog.clear() # 등록되는 데이터셋 이름이 안 겹치게 clear
  for d in ["train"]: 
    DatasetCatalog.register("port_" + d, lambda d=d: dataset+add_data+aug_dataset) #dataset
    MetadataCatalog.get("port_" + d).set(thing_classes=classes)
  port_metadata = MetadataCatalog.get("port_train")

  cfg.DATASETS.TRAIN = ("port_train") # 데이터셋 지정
  cfg.OUTPUT_DIR = dataset_dir+'output/' + self_model_dir + str(flag)
  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

  with open(dataset_dir+'output/' + self_model_dir + str(flag)+'/add_data_{}.pickle'.format(flag), 'wb') as f:
    pickle.dump(add_data, f)
  
  with open(dataset_dir+'output/' + self_model_dir + str(flag)+'/unlabel_{}.pickle'.format(flag), 'wb') as f:
    pickle.dump(unlabel, f)
  
  trainer = DefaultTrainer(cfg)  # 훈련, 모델 저장
  trainer.resume_or_load(resume=False)
  trainer.train()

  cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") # predictor 생성
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
  predictor = DefaultPredictor(cfg)   
  
  flag += 1 

  print("*********************** 데이터 추가 훈련 횟수: ",flag,"***********************")
  print("*********************** 세이브 완료! ***********************")

  if flag == 7 : 
    break

### Inference Set 
Colab의 한계로 세션이 끊겼을 때만 진행하는 부분입니다.

In [ ]:
with open('/content/drive/MyDrive/ai challenge/output/top12_mul2_step_self_5/add_data_5.pickle', 'rb') as f:
    add_data = pickle.load(f)

len(add_data)

756

In [ ]:
DatasetCatalog.clear() # 등록되는 데이터셋 이름이 안 겹치게 clear
for d in ["train"]: 
  DatasetCatalog.register("port_" + d, lambda d=d: add_data+aug_dataset+dataset) #dataset
  MetadataCatalog.get("port_" + d).set(thing_classes=classes)
port_metadata = MetadataCatalog.get("port_train")

cfg.DATASETS.TRAIN = ("port_train") # 데이터셋 지정

In [ ]:
#self_model_dir = "self"
#dataset_dir+'output/' + self_model_dir
cfg.OUTPUT_DIR = "/content/drive/MyDrive/ai challenge/output/top12_mul2_step_self_5"     

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") # predictor 생성
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
predictor = DefaultPredictor(cfg) 

[06/20 22:48:03 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                        | Shapes                                          |
|:------------------------------------------------|:-------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.res2.0.conv1.*                         | backbone.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.res2.0.conv2.*                         | backbone.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.res2.0.conv3.*                         | backbone.res2.0.conv3.{norm.bias,norm.running_mean,norm.running_var,norm.weight,w

### 세션이 끊기지 않으면 여기서부터 이어서 실행합니다.

In [ ]:
sub = pd.read_csv("/content/drive/MyDrive/sample_submission/sample_submission.csv")

# 초기화
sub['class'] = ""
sub['prediction'] = ""

# submission 파일 순서 지키기 위함

test_name = []

for i in range(len(sub)):
  test_name.append(dataset_dir+test_dir+sub['file_name'][i])

In [ ]:
from tqdm import tqdm

for idx ,test in tqdm(enumerate(test_name)):
  im = cv2.imread(test)
  outputs = predictor(im)  

  if len(outputs['instances'].pred_boxes) == 0:
    sub['class'][idx] = np.nan
    sub['prediction'][idx] = np.nan
  else:
    sub['class'][idx] = classes[outputs['instances'].pred_classes[0].tolist()] 
    sub['prediction'][idx] = mask_to_coordinates(outputs['instances'].pred_masks[0].cpu())

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
8409it [47:53,  2.93it/s]


In [ ]:
sub.isna().sum()

file_name      0
class         30
prediction    30
dtype: int64

In [ ]:
sub[sub['class'].isna()]['file_name']

413     31m47IZl9E.jpg
529     3uWYK7l0KG.jpg
860     6UJdL78mLj.jpg
1583    C8wdpBMODy.jpg
1693    Cu4C2ihGNT.jpg
1846    E3M0XY3EZk.jpg
2026    FPxUoOjyMd.jpg
2092    Frgr7uhGI5.jpg
2249    Gu6DPo3up1.jpg
2421    I6D37ltQKd.jpg
2712    KIhMDzaCAV.jpg
2879    LasmyzeDfR.jpg
2932    LwZczs6zjy.jpg
3448    PoNxrIrkX2.jpg
3499    QDAdtpiv0j.jpg
3934    TDfaU9KHRv.jpg
3968    TP3LEDzCwX.jpg
4217    VEWWDMNtcn.jpg
4931    aRI3KSkQXW.jpg
5385    dbM6CRM094.jpg
5948    hs28iXxtsb.jpg
6040    iV5Ld50ysI.jpg
6311    kOP5gLmeQr.jpg
6586    mZ8EBdFoU3.jpg
7172    qxhdFs6OWc.jpg
7197    r7tobfsxEE.jpg
7467    t9UJApCYIz.jpg
7669    uf3XWt93JY.jpg
7972    wx8PkqtjBS.jpg
8049    xTpiZNvhab.jpg
Name: file_name, dtype: object

In [ ]:
sub.fillna(method="ffill",inplace=True)
sub.isna().sum()

file_name     0
class         0
prediction    0
dtype: int64

In [ ]:
sub.to_csv("result.csv",index=False)